In [1]:
import keras
import numpy as np
import os
import random
import tensorflow as tf
import sys

from keras import backend as K
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate, Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import optimizers
from keras.models import model_from_json
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from keras.callbacks import ModelCheckpoint
from skimage.transform import resize
from skimage.io import imsave

print("Python version",sys.version)
print ("keras version", keras.__version__)
print ("tensorflow version", tf.__version__)
print ("numpy version", np.__version__)

c:\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Python version 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
keras version 2.1.4
tensorflow version 1.5.0
numpy version 1.14.2


In [2]:
# Get images
X = []
for filename in os.listdir('data/images/Train/'):
    X.append(img_to_array(load_img('data/images/Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

In [3]:
#Load weights
#inception = InceptionResNetV2(weights=None, include_top=True)
#inception.load_weights('inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.get_default_graph()

In [4]:
# load json and create model
json_file = open('model_colorme.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

optz = optimizers.Adam(lr=0.00001)
#optz = optimizers.rmsprop(lr=0.00001)
model.compile(optimizer=optz, loss='mse',metrics=['acc'])
model.summary()

# load weights into new model
#model.load_weights("model.h5")
model.load_weights("colorme_weight10.h5",by_name=True)
print("Loaded weights from checkpoint")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_323 (Conv2D)             (None, 128, 128, 64) 640         input_24[0][0]                   
__________________________________________________________________________________________________
conv2d_324 (Conv2D)             (None, 128, 128, 128 73856       conv2d_323[0][0]                 
__________________________________________________________________________________________________
conv2d_325 (Conv2D)             (None, 64, 64, 128)  147584      conv2d_324[0][0]                 
__________________________________________________________________________________________________
conv2d_326

In [5]:
filepath="weights-improvement-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1,save_best_only=True,mode='max')
callbacks_list = [checkpoint]

In [6]:
#Create embedding
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.4,
        zoom_range=0.4,
        rotation_range=40,
        horizontal_flip=True)

#Generate training data
batch_size = 5

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)

In [9]:
#Train model
model.fit_generator(image_a_b_gen(batch_size), epochs=100, steps_per_epoch=200, callbacks=callbacks_list, verbose=1)

# Save model
model_json = model.to_json()
with open("model_colorme.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("colorme_weight11.h5")

#Make predictions on validation images
color_me = []
for filename in os.listdir('data/images/Test/'):
    color_me.append(img_to_array(load_img('data/images/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

c:\anaconda3\envs\tensorflow\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
c:\anaconda3\envs\tensorflow\lib\site-packages\skimage\color\colorconv.py:985: UserWarning: Color data out of range: Z < 0 in 1 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
